In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pylab as plt
%pylab inline
import seaborn as sns
sns.set(style=('whitegrid'))

Populating the interactive namespace from numpy and matplotlib


C:\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# Import data

In [3]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_train_fr = pd.read_csv("train_fr.csv")
df_train_es = pd.read_csv("train_es.csv")
df_train_de = pd.read_csv("train_de.csv")

df_train = df_train.append(df_train_fr, ignore_index=True)
df_train = df_train.append(df_train_es, ignore_index=True)
df_train = df_train.append(df_train_de, ignore_index=True)

In [4]:
df_train = df_train.fillna("Na")
df_test = df_test.fillna("Na")

def text_preprocessing(text):
    from nltk.corpus import stopwords
    from nltk.tokenize import RegexpTokenizer
    from nltk.stem import WordNetLemmatizer
    
    tokenizer = RegexpTokenizer(r'\w+')
    stop = set(stopwords.words('english')) 
    lemmatizer = WordNetLemmatizer()
     
    processed = []   
    print("Start preprocessing: ")
    for i in range(len(text)):
        if i == 10:
            print("yehaa we making progress")
        if i%10000 == 0:
            print("{} of {}".format(i, len(text)))
        temp = tokenizer.tokenize(text.iloc[i])
        temp = [lemmatizer.lemmatize(i) for i in temp if i not in stop]
        temp = (" ").join(temp)
        processed.append(temp)
        
    return processed

df_train["processed"] = text_preprocessing(df_train["comment_text"])
df_test["processed"] = text_preprocessing(df_test["comment_text"])

# create submission file
submission_chains = pd.read_csv('sample_submission.csv')

# create a function to add features
def add_feature(X, feature_to_add):
    '''
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    '''
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

In [22]:
from sklearn.model_selection import train_test_split
import gc
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

df_train = df_train.fillna("unknown")
df_test = df_test.fillna("unknown")

comments_train, comments_valid, train_l, valid_l = train_test_split(df_train['comment_text'],df_train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]
                                                          , test_size=0.2, random_state=1337)



'''
def text_process(comment):
    nopunc = [char for char in comment if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

#Couldnt remove the stop words using the above function since it is taking too long
#Can try it on a local machine, I feel it improves the score-Not sure though



transform_com = CountVectorizer().fit(pd.concat([train['comment_text'],test['comment_text']],axis=0))
comments_train = transform_com.transform(train['comment_text'])
comments_test = transform_com.transform(test['comment_text'])
gc.collect()

#Using the tokenize function from Jeremy's kernel
#import re, string
#re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
#def tokenize(s): return re_tok.sub(r' \1 ', s).split()
'''
transform_com = TfidfVectorizer(ngram_range=(1,3), 
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1).fit(pd.concat([df_train['comment_text'],df_test['comment_text']],axis=0))

comments_train = transform_com.transform(comments_train)
comments_valid = transform_com.transform(comments_valid)
comments_test = transform_com.transform(df_test['comment_text'])

gc.collect()

"""
import lightgbm as lgb

def runlgb()
    # create dataset for lightgbm
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
    
    # specify your configurations as a dict
    params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
"""
import xgboost as xgb

def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=2, num_rounds=400):
    param = {}
    param['objective'] = 'binary:logistic'
    param['eta'] = 0.1
    param['max_depth'] = 5
    param['silent'] = 1
    param['eval_metric'] = 'logloss'
    param['min_child_weight'] = 1
    param['subsample'] = 0.4
    param['colsample_bytree'] = 0.6
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)
    xgtest = xgb.DMatrix(test_X)
    model = xgb.train(plst, xgtrain, num_rounds)
    pred_test_y = model.predict(xgtest)
    return pred_test_y, model



def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=1337, num_rounds=400):
    param = {}
    param['objective'] = 'binary:logistic'
    param['eta'] = 0.12
    param['max_depth'] = 5
    param['silent'] = 1
    param['eval_metric'] = 'logloss'
    param['min_child_weight'] = 1
    param['subsample'] = 0.5
    param['colsample_bytree'] = 0.7
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest)
    return model
    

col = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
preds = np.zeros((df_test.shape[0], len(col)))

for i, j in enumerate(col):
    print('fit '+j)
    model = runXGB(comments_train, train_l[j], comments_valid,valid_l[j])
    preds[:,i] = model.predict(xgb.DMatrix(comments_test))
#    train_X = add_feature(train_X, train_y[j])
#    xgtrain
    gc.collect()

#subm = pd.read_csv('sample_submission.csv')    
#submid = pd.DataFrame({'id': subm["id"]})
#submission = pd.concat([submid, pd.DataFrame(preds, columns = col)], axis=1)
#submission.to_csv('xgb_remote.csv', index=False)


fit toxic
[0]	train-logloss:0.609551	test-logloss:0.609441
Multiple eval metrics have been passed: 'test-logloss' will be used for early stopping.

Will train until test-logloss hasn't improved in 20 rounds.
[1]	train-logloss:0.543296	test-logloss:0.543099
[2]	train-logloss:0.491522	test-logloss:0.491231
[3]	train-logloss:0.448993	test-logloss:0.448637
[4]	train-logloss:0.412603	test-logloss:0.412217
[5]	train-logloss:0.382788	test-logloss:0.382392
[6]	train-logloss:0.357348	test-logloss:0.356808
[7]	train-logloss:0.335549	test-logloss:0.334941
[8]	train-logloss:0.317461	test-logloss:0.316801
[9]	train-logloss:0.301955	test-logloss:0.301216
[10]	train-logloss:0.288542	test-logloss:0.287959
[11]	train-logloss:0.277253	test-logloss:0.276583
[12]	train-logloss:0.267193	test-logloss:0.266573
[13]	train-logloss:0.258786	test-logloss:0.258143
[14]	train-logloss:0.251362	test-logloss:0.250703
[15]	train-logloss:0.244659	test-logloss:0.244109
[16]	train-logloss:0.239039	test-logloss:0.238505
[

[161]	train-logloss:0.136148	test-logloss:0.140974
[162]	train-logloss:0.135966	test-logloss:0.140818
[163]	train-logloss:0.135793	test-logloss:0.140662
[164]	train-logloss:0.135615	test-logloss:0.140488
[165]	train-logloss:0.135366	test-logloss:0.140286
[166]	train-logloss:0.135176	test-logloss:0.140108
[167]	train-logloss:0.135004	test-logloss:0.139929
[168]	train-logloss:0.134781	test-logloss:0.139723
[169]	train-logloss:0.134598	test-logloss:0.139545
[170]	train-logloss:0.134395	test-logloss:0.139363
[171]	train-logloss:0.134234	test-logloss:0.139236
[172]	train-logloss:0.134054	test-logloss:0.139092
[173]	train-logloss:0.133891	test-logloss:0.138947
[174]	train-logloss:0.13372	test-logloss:0.138805
[175]	train-logloss:0.133552	test-logloss:0.138642
[176]	train-logloss:0.133365	test-logloss:0.138458
[177]	train-logloss:0.133179	test-logloss:0.138295
[178]	train-logloss:0.133022	test-logloss:0.138177
[179]	train-logloss:0.132812	test-logloss:0.138022
[180]	train-logloss:0.132629	tes

[323]	train-logloss:0.114938	test-logloss:0.123015
[324]	train-logloss:0.114852	test-logloss:0.122934
[325]	train-logloss:0.114747	test-logloss:0.122845
[326]	train-logloss:0.114652	test-logloss:0.122769
[327]	train-logloss:0.114568	test-logloss:0.122727
[328]	train-logloss:0.114487	test-logloss:0.122661
[329]	train-logloss:0.114373	test-logloss:0.122568
[330]	train-logloss:0.114275	test-logloss:0.122491
[331]	train-logloss:0.11421	test-logloss:0.122425
[332]	train-logloss:0.114132	test-logloss:0.122371
[333]	train-logloss:0.114053	test-logloss:0.122306
[334]	train-logloss:0.113975	test-logloss:0.122234
[335]	train-logloss:0.113906	test-logloss:0.122184
[336]	train-logloss:0.113832	test-logloss:0.122082
[337]	train-logloss:0.113761	test-logloss:0.122029
[338]	train-logloss:0.113679	test-logloss:0.121969
[339]	train-logloss:0.11357	test-logloss:0.121868
[340]	train-logloss:0.1135	test-logloss:0.121813
[341]	train-logloss:0.113438	test-logloss:0.121772
[342]	train-logloss:0.113356	test-l

[83]	train-logloss:0.023876	test-logloss:0.026936
[84]	train-logloss:0.023819	test-logloss:0.026909
[85]	train-logloss:0.023746	test-logloss:0.026848
[86]	train-logloss:0.023677	test-logloss:0.026795
[87]	train-logloss:0.023591	test-logloss:0.026706
[88]	train-logloss:0.023522	test-logloss:0.026643
[89]	train-logloss:0.023458	test-logloss:0.026608
[90]	train-logloss:0.023386	test-logloss:0.026564
[91]	train-logloss:0.02332	test-logloss:0.026502
[92]	train-logloss:0.02326	test-logloss:0.026456
[93]	train-logloss:0.023209	test-logloss:0.02643
[94]	train-logloss:0.023128	test-logloss:0.02641
[95]	train-logloss:0.023074	test-logloss:0.026349
[96]	train-logloss:0.023029	test-logloss:0.026325
[97]	train-logloss:0.022967	test-logloss:0.026266
[98]	train-logloss:0.022913	test-logloss:0.026244
[99]	train-logloss:0.022856	test-logloss:0.026234
[100]	train-logloss:0.022817	test-logloss:0.026205
[101]	train-logloss:0.022782	test-logloss:0.026174
[102]	train-logloss:0.022743	test-logloss:0.026127
[

[245]	train-logloss:0.018636	test-logloss:0.023479
[246]	train-logloss:0.018617	test-logloss:0.023462
[247]	train-logloss:0.018595	test-logloss:0.023456
[248]	train-logloss:0.018579	test-logloss:0.023435
[249]	train-logloss:0.018548	test-logloss:0.023419
[250]	train-logloss:0.018527	test-logloss:0.023406
[251]	train-logloss:0.018507	test-logloss:0.023391
[252]	train-logloss:0.018484	test-logloss:0.023369
[253]	train-logloss:0.018468	test-logloss:0.023364
[254]	train-logloss:0.018453	test-logloss:0.023362
[255]	train-logloss:0.018431	test-logloss:0.023342
[256]	train-logloss:0.018408	test-logloss:0.023331
[257]	train-logloss:0.018389	test-logloss:0.023313
[258]	train-logloss:0.018368	test-logloss:0.023305
[259]	train-logloss:0.018342	test-logloss:0.023297
[260]	train-logloss:0.018326	test-logloss:0.023286
[261]	train-logloss:0.018312	test-logloss:0.02328
[262]	train-logloss:0.018286	test-logloss:0.023271
[263]	train-logloss:0.01827	test-logloss:0.023262
[264]	train-logloss:0.018247	test

...

[11]	train-logloss:0.204165	test-logloss:0.204375
[12]	train-logloss:0.192189	test-logloss:0.192458
[13]	train-logloss:0.181953	test-logloss:0.18222
[14]	train-logloss:0.172994	test-logloss:0.173247
[15]	train-logloss:0.165155	test-logloss:0.165443
[16]	train-logloss:0.158304	test-logloss:0.158706
[17]	train-logloss:0.152385	test-logloss:0.152833
[18]	train-logloss:0.14717	test-logloss:0.147591
[19]	train-logloss:0.142505	test-logloss:0.143005
[20]	train-logloss:0.138373	test-logloss:0.138971
[21]	train-logloss:0.134858	test-logloss:0.135511
[22]	train-logloss:0.131806	test-logloss:0.132444
[23]	train-logloss:0.128949	test-logloss:0.129704
[24]	train-logloss:0.126404	test-logloss:0.127211
[25]	train-logloss:0.124063	test-logloss:0.124859
[26]	train-logloss:0.121864	test-logloss:0.122753
[27]	train-logloss:0.120001	test-logloss:0.121019
[28]	train-logloss:0.118298	test-logloss:0.119387
[29]	train-logloss:0.116714	test-logloss:0.117818
[30]	train-logloss:0.115269	test-logloss:0.116443
[3

[174]	train-logloss:0.074241	test-logloss:0.079618
[175]	train-logloss:0.074157	test-logloss:0.079541
[176]	train-logloss:0.074071	test-logloss:0.079479
[177]	train-logloss:0.073982	test-logloss:0.079405
[178]	train-logloss:0.073883	test-logloss:0.079341
[179]	train-logloss:0.07381	test-logloss:0.079284
[180]	train-logloss:0.073708	test-logloss:0.079219
[181]	train-logloss:0.073631	test-logloss:0.079159
[182]	train-logloss:0.073553	test-logloss:0.079084
[183]	train-logloss:0.073426	test-logloss:0.07896
[184]	train-logloss:0.073343	test-logloss:0.078889
[185]	train-logloss:0.07325	test-logloss:0.078811
[186]	train-logloss:0.073181	test-logloss:0.078762
[187]	train-logloss:0.073084	test-logloss:0.078684
[188]	train-logloss:0.072988	test-logloss:0.078613
[189]	train-logloss:0.07288	test-logloss:0.078539
[190]	train-logloss:0.072786	test-logloss:0.078438
[191]	train-logloss:0.072675	test-logloss:0.078374
[192]	train-logloss:0.072584	test-logloss:0.078322
[193]	train-logloss:0.072514	test-l

[336]	train-logloss:0.063611	test-logloss:0.071783
[337]	train-logloss:0.063567	test-logloss:0.071752
[338]	train-logloss:0.06353	test-logloss:0.071729
[339]	train-logloss:0.063499	test-logloss:0.071698
[340]	train-logloss:0.06346	test-logloss:0.071686
[341]	train-logloss:0.063409	test-logloss:0.071654
[342]	train-logloss:0.063356	test-logloss:0.071636
[343]	train-logloss:0.063315	test-logloss:0.071608
[344]	train-logloss:0.063275	test-logloss:0.071589
[345]	train-logloss:0.063233	test-logloss:0.071558
[346]	train-logloss:0.063196	test-logloss:0.071533
[347]	train-logloss:0.06316	test-logloss:0.071522
[348]	train-logloss:0.063103	test-logloss:0.071485
[349]	train-logloss:0.063055	test-logloss:0.071454
[350]	train-logloss:0.063011	test-logloss:0.071442
[351]	train-logloss:0.062977	test-logloss:0.0714
[352]	train-logloss:0.062933	test-logloss:0.071368
[353]	train-logloss:0.062883	test-logloss:0.071348
[354]	train-logloss:0.062841	test-logloss:0.071308
[355]	train-logloss:0.062807	test-lo

[97]	train-logloss:0.022219	test-logloss:0.024422
[98]	train-logloss:0.022139	test-logloss:0.024362
[99]	train-logloss:0.022063	test-logloss:0.024275
[100]	train-logloss:0.022011	test-logloss:0.024225
[101]	train-logloss:0.021963	test-logloss:0.024196
[102]	train-logloss:0.021893	test-logloss:0.024131
[103]	train-logloss:0.021789	test-logloss:0.024071
[104]	train-logloss:0.0217	test-logloss:0.023994
[105]	train-logloss:0.021621	test-logloss:0.023934
[106]	train-logloss:0.021569	test-logloss:0.02389
[107]	train-logloss:0.021513	test-logloss:0.023849
[108]	train-logloss:0.021443	test-logloss:0.02379
[109]	train-logloss:0.021398	test-logloss:0.023733
[110]	train-logloss:0.021343	test-logloss:0.023675
[111]	train-logloss:0.021252	test-logloss:0.023619
[112]	train-logloss:0.021175	test-logloss:0.023552
[113]	train-logloss:0.021107	test-logloss:0.023499
[114]	train-logloss:0.021039	test-logloss:0.02343
[115]	train-logloss:0.020993	test-logloss:0.02338
[116]	train-logloss:0.020907	test-loglos

[259]	train-logloss:0.015951	test-logloss:0.019613
[260]	train-logloss:0.015926	test-logloss:0.019592
[261]	train-logloss:0.015902	test-logloss:0.019576
[262]	train-logloss:0.015881	test-logloss:0.01956
[263]	train-logloss:0.015857	test-logloss:0.019549
[264]	train-logloss:0.015833	test-logloss:0.019524
[265]	train-logloss:0.015812	test-logloss:0.0195
[266]	train-logloss:0.015786	test-logloss:0.019476
[267]	train-logloss:0.015767	test-logloss:0.019461
[268]	train-logloss:0.015747	test-logloss:0.019435
[269]	train-logloss:0.015726	test-logloss:0.01942
[270]	train-logloss:0.015699	test-logloss:0.01941
[271]	train-logloss:0.01568	test-logloss:0.019404
[272]	train-logloss:0.015656	test-logloss:0.019394
[273]	train-logloss:0.015638	test-logloss:0.019384
[274]	train-logloss:0.015622	test-logloss:0.019369
[275]	train-logloss:0.015604	test-logloss:0.019349
[276]	train-logloss:0.015592	test-logloss:0.019344
[277]	train-logloss:0.015576	test-logloss:0.019338
[278]	train-logloss:0.015556	test-log

In [6]:
import scipy as sp

def save_sparse_csr(filename,array):
    np.savez(filename,data = array.data ,indices=array.indices,
             indptr =array.indptr, shape=array.shape )

def load_sparse_csr(filename):
    loader = np.load(filename)
    return sp.sparse.csr_matrix((loader['data'], loader['indices'], loader['indptr']), shape = loader['shape'])

#save_sparse_csr("comments_train", comments_train)
#save_sparse_csr("comments_valid", comments_valid)
#save_sparse_csr("comments_test", comments_test)

In [10]:
from sklearn.model_selection import train_test_split
import gc
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb


df_train = df_train.fillna("unknown")
df_test = df_test.fillna("unknown")

comments_train, comments_valid, train_l, valid_l = train_test_split(df_train['comment_text'],df_train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]
                                                          , test_size=0.2, random_state=1337)
"""
transform_com = TfidfVectorizer(ngram_range=(1,3), 
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1).fit(pd.concat([df_train['comment_text'],df_test['comment_text']],axis=0))

comments_train = transform_com.transform(comments_train)
comments_valid = transform_com.transform(comments_valid)
comments_test = transform_com.transform(df_test['comment_text'])

gc.collect()
"""
comments_train = load_sparse_csr("comments_train.npz")
comments_valid = load_sparse_csr("comments_valid.npz")
comments_test = load_sparse_csr("comments_test.npz")

def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=1337, num_rounds=400):
    param = {}
    param['objective'] = 'binary:logistic'
    param['eta'] = 0.12
    param['max_depth'] = 5
    param['silent'] = 1
    param['eval_metric'] = 'logloss'
    param['min_child_weight'] = 1
    param['subsample'] = 0.5
    param['colsample_bytree'] = 0.7
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest)
    return model
    

col = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
preds = np.zeros((df_test.shape[0], len(col)))

for i, j in enumerate(col):
    print('fit '+j)
    model = runXGB(comments_train, train_l[j], comments_valid,valid_l[j])
    preds[:,i] = model.predict(xgb.DMatrix(comments_test))
    #to_add = train_l[j].values.reshape(train_l.shape[0], 1)
    #comments_train = sp.sparse.hstack((comments_train, to_add))
    gc.collect()

#subm = pd.read_csv('sample_submission.csv')    
#submid = pd.DataFrame({'id': subm["id"]})
#submission = pd.concat([submid, pd.DataFrame(preds, columns = col)], axis=1)
#submission.to_csv('xgb_remote.csv', index=False)


fit toxic
[0]	train-logloss:0.609551	test-logloss:0.609441
Multiple eval metrics have been passed: 'test-logloss' will be used for early stopping.

Will train until test-logloss hasn't improved in 20 rounds.
[1]	train-logloss:0.543296	test-logloss:0.543099
[2]	train-logloss:0.491522	test-logloss:0.491231
[3]	train-logloss:0.448993	test-logloss:0.448637
[4]	train-logloss:0.412603	test-logloss:0.412217
[5]	train-logloss:0.382788	test-logloss:0.382392
[6]	train-logloss:0.357348	test-logloss:0.356808
[7]	train-logloss:0.335549	test-logloss:0.334941
[8]	train-logloss:0.317461	test-logloss:0.316801
[9]	train-logloss:0.301955	test-logloss:0.301216
[10]	train-logloss:0.288542	test-logloss:0.287959
[11]	train-logloss:0.277253	test-logloss:0.276583
[12]	train-logloss:0.267193	test-logloss:0.266573
[13]	train-logloss:0.258786	test-logloss:0.258143
[14]	train-logloss:0.251362	test-logloss:0.250703
[15]	train-logloss:0.244659	test-logloss:0.244109
[16]	train-logloss:0.239039	test-logloss:0.238505
[

[161]	train-logloss:0.136148	test-logloss:0.140974
[162]	train-logloss:0.135966	test-logloss:0.140818
[163]	train-logloss:0.135793	test-logloss:0.140662
[164]	train-logloss:0.135615	test-logloss:0.140488
[165]	train-logloss:0.135366	test-logloss:0.140286
[166]	train-logloss:0.135176	test-logloss:0.140108
[167]	train-logloss:0.135004	test-logloss:0.139929
[168]	train-logloss:0.134781	test-logloss:0.139723
[169]	train-logloss:0.134598	test-logloss:0.139545
[170]	train-logloss:0.134395	test-logloss:0.139363
[171]	train-logloss:0.134234	test-logloss:0.139236
[172]	train-logloss:0.134054	test-logloss:0.139092
[173]	train-logloss:0.133891	test-logloss:0.138947
[174]	train-logloss:0.13372	test-logloss:0.138805
[175]	train-logloss:0.133552	test-logloss:0.138642
[176]	train-logloss:0.133365	test-logloss:0.138458
[177]	train-logloss:0.133179	test-logloss:0.138295
[178]	train-logloss:0.133022	test-logloss:0.138177
[179]	train-logloss:0.132812	test-logloss:0.138022
[180]	train-logloss:0.132629	tes

[323]	train-logloss:0.114938	test-logloss:0.123015
[324]	train-logloss:0.114852	test-logloss:0.122934
[325]	train-logloss:0.114747	test-logloss:0.122845
[326]	train-logloss:0.114652	test-logloss:0.122769
[327]	train-logloss:0.114568	test-logloss:0.122727
[328]	train-logloss:0.114487	test-logloss:0.122661
[329]	train-logloss:0.114373	test-logloss:0.122568
[330]	train-logloss:0.114275	test-logloss:0.122491
[331]	train-logloss:0.11421	test-logloss:0.122425
[332]	train-logloss:0.114132	test-logloss:0.122371
[333]	train-logloss:0.114053	test-logloss:0.122306
[334]	train-logloss:0.113975	test-logloss:0.122234
[335]	train-logloss:0.113906	test-logloss:0.122184
[336]	train-logloss:0.113832	test-logloss:0.122082
[337]	train-logloss:0.113761	test-logloss:0.122029
[338]	train-logloss:0.113679	test-logloss:0.121969
[339]	train-logloss:0.11357	test-logloss:0.121868
[340]	train-logloss:0.1135	test-logloss:0.121813
[341]	train-logloss:0.113438	test-logloss:0.121772
[342]	train-logloss:0.113356	test-l

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [77]:
comments_test = sp.sparse.hstack((comments_train, a))
#train_l.values.shape
#a = (train_l["toxic"].values.reshape(train_l.shape[0],1))
#train_l[j].shape

In [11]:
model = runXGB(comments_train, train_l[j], comments_valid,valid_l[j])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [64]:
print(comments_train.shape)
printdasd(comments_test.shape)
print(comments_valid.shape)

print(train_l.values.shape)

(510627, 5492599)
(153164, 5492599)
(127657, 5492599)
(510627, 6)


In [39]:
preds2 = preds.copy()

In [ ]:
 #logit() the predictions, subtract 0.5, and expit() 
from scipy.special import logit,expit
test = np.zeros((df_test.shape[0], len(col)))
for i in range(len(col)):
    test[:,i] = logit(preds[:,i])
    test[:,i] -= 0.7
    test[:,i] = expit(test[:,i])

In [ ]:
subm = pd.read_csv('sample_submission.csv')    
submid = pd.DataFrame({'id': subm["id"]})
submission = pd.concat([submid, pd.DataFrame(test, columns = col)], axis=1)
submission.to_csv('xgb_remote5.csv', index=False)

In [30]:
model.save_model('0001.model')